In [ ]:
import os
import glob
import numpy as np
import pandas as pd

from decimal import *

getcontext().prec = 33

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['legend.frameon']=False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif']=['Helvetica']
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['axes.axisbelow'] = True
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['patch.linewidth'] = 1
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize']=(12,10)

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

from scipy.stats import lognorm
from scipy.constants import pi, g, gas_constant
from scipy.optimize import least_squares
from scipy.optimize import differential_evolution
from scipy.linalg import svd

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# read file contents into a dataframe

In [ ]:
def readFileContents(filenames, minChord, chordIncrement, nClasses):
    """
    reads file contents into a pandas data frame
    """
    class_dict = {}
    for fname in filenames:
        # remove path from file name
        _, f = os.path.split(fname)
            
        # remove file extension from file name obtained above
        key, _ = os.path.splitext(f)
            
        # read file contents into a data frame
        df = pd.read_csv(fname, delimiter='\t')
        
        # bin data into different size groups
        class_dict[key] = createClasses(df, minChord, chordIncrement, nClasses)
        
    return class_dict

# create bubble classes

In [ ]:
def createClasses(df, minChord, chordIncrement, nClasses):
    """
    Categorizes bubbles into size classes
    
    args:
        df: pandas dataframe containing the dataset
        
    return:
        arr: numpy array containing class averages
    """
    rows = len(df)

    count = np.zeros(nClasses)
    cbr_num = np.zeros(nClasses)
    ubr_num = np.zeros(nClasses)
    for i in range(rows):
        cbri = df['Chord(mm)'][i]
        ubri = df['Velocity(m/s)'][i]
        for j in range(nClasses):
            lowerEnd = minChord + j*chordIncrement 
            upperEnd = minChord + (j+1)*chordIncrement
            if(cbri > lowerEnd and cbri < upperEnd):
                count[j] = count[j] + 1
                cbr_num[j] = cbr_num[j] + cbri
                ubr_num[j] = ubr_num[j] + ubri
                
    classes = np.zeros((nClasses, 3))
    
    for k in range(nClasses):
        classes[k,0] = cbr_num[k]/count[k]
        classes[k,1] = ubr_num[k]/count[k]
        classes[k,2] = count[k]
    
    return classes

# radially average bubble properties

In [ ]:
def radialAverage(class_dict, P, eg_real, eg_folder, UL, rhog, rhol, mul, sigma, nClasses, beta0, beta1):
    R = 0.50*0.1016
    rR = ['-0.80', '-0.60', '-0.40', '-0.20', '0', '0.20', '0.40', '0.60', '0.80']
    
    A0 = 0.50*pi*(R**2 - (-0.80*R)**2)
    A1 = 0.50*pi*((-0.80*R)**2 - (-0.60*R)**2)
    A2 = 0.50*pi*((-0.60*R)**2 - (-0.40*R)**2)
    A3 = 0.50*pi*((-0.40*R)**2 - (-0.20*R)**2)
    A4 = 0.50*pi*((-0.20*R)**2 - (0.00*R)**2)
    A5 = 0.50*pi*((0.20*R)**2 - (0.0*R)**2)
    A6 = 0.50*pi*((0.40*R)**2 - (0.20*R)**2)
    A7 = 0.50*pi*((0.60*R)**2 - (0.40*R)**2)
    A8 = 0.50*pi*((0.80*R)**2 - (0.60*R)**2)
    
    A = [A0, A1, A2, A3, A4, A5, A6, A7, A8]
    
    Cbnums = np.zeros(nClasses)
    Cbdens = np.zeros(nClasses)
    Ubnums = np.zeros(nClasses)
    Ubdens = np.zeros(nClasses)
    
    for i in range(9):
        key = 'P' + str(P) + '-eG' + str(eg_folder) + '-rR' + rR[i]
        arr = class_dict[key]
        for j in range(nClasses):
            Cbnums[j] = Cbnums[j] + (arr[j,0]**3.0)*arr[j,2]*A[i]
            Cbdens[j] = Cbdens[j] + arr[j,2]*A[i]
            Ubnums[j] = Ubnums[j] + arr[j,1]*(arr[j,0]**3.0)*arr[j,2]*A[i]
            Ubdens[j] = Ubdens[j] + (arr[j,0]**3.0)*arr[j,2]*A[i]
    
    results = np.zeros([nClasses,3])
    for k in range(nClasses):
        Ubi = Ubnums[k]/Ubdens[k]
        Usi = Ubi - UL/(1.0 - eg_real)
        results[k,1] = Usi
        
        Cbi = (Cbnums[k]/Cbdens[k])**(1.0/3.0)
        dbi = convertChordToDiameter(Cbi, Usi, rhog, rhol, mul, sigma, beta0, beta1)
        results[k,0] = dbi
        
        Re = (rhol*Usi*0.001*dbi)/mul
        Eo = (g*(rhol - rhog)*(0.001*dbi)**2.0)/sigma
        #E = Wellek(Eo)
        #E = BesagniDeen(Re, Eo)
        #E = BesagniDeenRevised(Re, Eo, beta0, beta1)
        E = BesagniDeenRevised(Re, Eo, beta0, beta1)
        Cdi = (4.0/3.0)*(rhol-rhog)/rhol*(g*(0.001*dbi)*E**(2.0/3.0))/(Usi**2.0)
        results[k,2] = Cdi

    return results

# convert bubble chord length to diameter

In [ ]:
def convertChordToDiameter(cbi, usi, rhog, rhol, mul, sigma, beta0, beta1):
    """
    converts bubble chord length to volume equivalent diameter
    
    args:
        cbi: bubble chord length
        usi: bubble slip velocity
        rhog: gas density
        rhol: liquid density
        mul: liquid viscosity
        sigma: surface tension
        
    returns:
        dbi: volume equivalent spherical diameter
    """
    Re = (rhol*usi*0.0015*cbi)/mul
    Eo = ((rhol - rhog)*g*(0.0015*cbi)**2)/sigma
    #E = Wellek(Eo)
    #E = BesagniDeen(Re, Eo)
    #E = BesagniDeenRevised(Re, Eo, beta0, beta1)
    E = BesagniDeenRevised(Re, Eo, beta0, beta1)
    
    dbi = 3.0/2.0*cbi*E**(-2.0/3.0)
    
    relError = 1.0
    relTol = 1.0e-8
    
    while(relError > relTol):
        dbi_prev = dbi
        Re = (rhol*usi*0.001*dbi_prev)/mul
        Eo = ((rhol - rhog)*g*(0.001*dbi_prev)**2)/sigma
        #E = Wellek(Eo)
        #E = BesagniDeen(Re, Eo)
        E = BesagniDeenRevised(Re, Eo, beta0, beta1)
        dbi = 3.0/2.0*cbi*E**(-2.0/3.0)
        relError = abs((dbi - dbi_prev)/dbi)
        
    return dbi

# aspect ratio models

In [ ]:
def Wellek(Eo):
    E = 1.0/(1.0 + 0.163*Eo**0.757)
    return E

In [ ]:
def BesagniDeen(Re, Eo):
    E = 1.0/((1.0 + 0.45*(Re*Eo))**0.08)
    return E

In [ ]:
def BesagniDeenRevised(Re, Eo, beta0, beta1):
    E = 1.0/(1.0 + beta0*(Re*Eo)**beta1)
    return E

# generates modelling data

In [ ]:
def modellingData(beta0, beta1):
    # main directory
    cwd = os.getcwd()

    # actual gas holdups
    eg_real = [0.092, 0.102, 0.098, 0.104, 
               0.178, 0.190, 0.187, 0.194,
               0.271, 0.242, 0.264, 0.279, 
               0.335, 0.324, 0.331, 0.342]

    # For locating gas holdup folders (mean values)
    eg_folder = [0.0987, 0.1873, 0.264, 0.3329]

    # actual operating pressures (MPa)
    # first row for the lowest mean gas holdup
    # last row for the highest mean gas holdup
    p_real = [0.14, 0.43, 2.0, 4.2,
              0.14, 0.46, 2.0, 4.0, 
              0.14, 0.46, 2.1, 3.9,
              0.14, 0.48, 2.1, 3.8]

    # For locating pressure folders (mean values)
    p_folder = [0.14, 0.46, 2, 4]

    # actual operating temperatures (K)
    # first row for the lowest mean gas holdup
    # last row for the highest mean gas holdup
    temp = [300.45, 300.235, 301.290, 303.340, 
            300.96, 300.29, 302.20, 303.853, 
            301.50, 300.66, 302.93, 304.137, 
            301.63, 299.543, 303.340, 304.608]

    # class information
    minChord = 0.00
    maxChord = 3.00
    chordIncrement = 0.05
    nClasses = int(round((maxChord - minChord)/chordIncrement))

    # liquid density
    rhol = 998.0
    # liquid viscosity
    mul = 0.000935
    # surface tension
    sigma = 0.068
    # liquid superficial velocity
    UL = 0.03
    
    # for external plotting
    drag_data = pd.DataFrame(data=None)
    # for storing class data
    bins = {}
    # for storing modelling data
    m_data = []
    mdict = {}
    
    # number of gas holdups
    nGasHoldups = len(eg_folder)
    # number of pressures
    nPressures = len(p_folder)
    
    for i in range(nGasHoldups):
        for j in range(nPressures):
            # actual gas holdup
            eg = eg_real[4*i + j]
            # actual temperature
            T = temp[4*i + j]
            # actual pressure
            p = p_real[4*i + j]
            # gas density
            rhog = (p*1e6*0.028)/(gas_constant*T)
            # path to files
            path_to_files = cwd + '/P = ' + str(p_folder[j]) + ' MPa/eG = ' + str(eg_folder[i])
            # obtain names of files in the directory specified above
            file_names = glob.glob(os.path.join(path_to_files, '*.txt'))
            # read file contents and bin the data
            bins = readFileContents(file_names, minChord, chordIncrement, nClasses)
            # radially average local bubble properties
            # also compute individual drag coefficients
            global_averages = radialAverage(bins, p_folder[j], eg, eg_folder[i], UL, rhog, rhol, mul, sigma, nClasses, beta0, beta1)
            # plot global data
            dia = global_averages[:,0]
            drag_coeff = global_averages[:,2]
        
            # store radially averaged data for external plotting
            keystring = 'P'+str(int(1000*p_folder[j]))+'-eG' + str(int(10000*eg_folder[i]))
            drag_data[keystring + '-dbi'] = global_averages[:,0]
            drag_data[keystring + '-Usi'] = global_averages[:,1]
            drag_data[keystring + '-Cdi'] = global_averages[:,2]
            drag_data[keystring + '-pPrime']=(p/0.10)*np.ones(nClasses)
            drag_data[keystring + '-eg'] = eg*np.ones(nClasses)
            drag_data[keystring + '-Re'] = (rhol*global_averages[:,1]*0.001*global_averages[:,0])/0.000935
            drag_data[keystring + '-Eo'] = ((rhol - rhog)*g*((0.001*global_averages[:,0])**2.0))/sigma
    
            # modelling data  
            Re = (rhol*global_averages[:,1]*0.001*global_averages[:,0])/0.000935
            Eo = ((rhol - rhog)*g*((0.001*global_averages[:,0])**2.0))/sigma
            pNorm = p/(0.1*Eo)
            mdict['Re'] = Re
            mdict['Eo'] = Eo
            mdict['eg'] = eg*np.ones(nClasses)
            mdict['p_norm'] = pNorm
            mdict['Cdi'] = global_averages[:,2]
            mdf = pd.DataFrame(mdict)
            m_data.append(mdf)
        
    # export all data to text files
    model_data = pd.concat(m_data, ignore_index=True)
    model_data.to_csv('model_data.txt', sep='\t', index=False, na_rep='nan')
    drag_data.to_csv('drag_data.txt', sep='\t', index=False, na_rep='nan')
    
    return model_data

In [ ]:
def plotModellingData(beta0, beta1):
    # main directory
    cwd = os.getcwd()

    # actual gas holdups
    eg_real = [0.092, 0.102, 0.098, 0.104, 
               0.178, 0.190, 0.187, 0.194,
               0.271, 0.242, 0.264, 0.279, 
               0.335, 0.324, 0.331, 0.342]

    # For locating gas holdup folders (mean values)
    eg_folder = [0.0987, 0.1873, 0.264, 0.3329]

    # actual operating pressures (MPa)
    # first row for the lowest mean gas holdup
    # last row for the highest mean gas holdup
    p_real = [0.14, 0.43, 2.0, 4.2,
              0.14, 0.46, 2.0, 4.0, 
              0.14, 0.46, 2.1, 3.9,
              0.14, 0.48, 2.1, 3.8]

    # For locating pressure folders (mean values)
    p_folder = [0.14, 0.46, 2, 4]

    # actual operating temperatures (K)
    # first row for the lowest mean gas holdup
    # last row for the highest mean gas holdup
    temp = [300.45, 300.235, 301.290, 303.340, 
            300.96, 300.29, 302.20, 303.853, 
            301.50, 300.66, 302.93, 304.137, 
            301.63, 299.543, 303.340, 304.608]

    # class information
    minChord = 0.20
    maxChord = 3.0
    chordIncrement = 0.05
    nClasses = int(round((maxChord - minChord)/chordIncrement))

    # liquid density
    rhol = 998.0
    # liquid viscosity
    mul = 0.000935
    # surface tension
    sigma = 0.068
    # liquid superficial velocity
    UL = 0.03
    
    # for external plotting
    drag_data = pd.DataFrame(data=None)
    # for storing class data
    bins = {}
    # for storing modelling data
    m_data = []
    mdict = {}
    
    # number of gas holdups
    nGasHoldups = len(eg_folder)
    # number of pressures
    nPressures = len(p_folder)
    markers = ['bo', 'rs', 'k^', 'gd']
    plt.rcParams['figure.figsize'] = (12,10)
    for i in range(nGasHoldups):
        plt.subplot(2,2,i+1)
        for j in range(nPressures):
            # actual gas holdup
            eg = eg_real[4*i + j]
            # actual temperature
            T = temp[4*i + j]
            # actual pressure
            p = p_real[4*i + j]
            # gas density
            rhog = (p*1e6*0.028)/(gas_constant*T)
            # path to files
            path_to_files = cwd + '/P = ' + str(p_folder[j]) + ' MPa/eG = ' + str(eg_folder[i])
            # obtain names of files in the directory specified above
            file_names = glob.glob(os.path.join(path_to_files, '*.txt'))
            # read file contents and bin the data
            bins = readFileContents(file_names, minChord, chordIncrement, nClasses)
            # radially average local bubble properties
            # also compute individual drag coefficients
            global_averages = radialAverage(bins, p_folder[j], eg, eg_folder[i], UL, rhog, rhol, mul, sigma, nClasses, beta0, beta1)
            # plot global data
            dia = global_averages[:,0]
            drag_coeff = global_averages[:,2]
            plt.plot(dia, drag_coeff, markers[j], label = "P = " + str(p) + "MPa") 
            # store radially averaged data for external plotting
            keystring = 'P'+str(int(1000*p_folder[j]))+'-eG' + str(int(10000*eg_folder[i]))
            drag_data[keystring + '-dbi'] = global_averages[:,0]
            drag_data[keystring + '-Usi'] = global_averages[:,1]
            drag_data[keystring + '-Cdi'] = global_averages[:,2]
            drag_data[keystring + '-pPrime']=(p/0.10)*np.ones(nClasses)
            drag_data[keystring + '-eg'] = eg*np.ones(nClasses)
            drag_data[keystring + '-Re'] = (rhol*global_averages[:,1]*0.001*global_averages[:,0])/0.000935
            drag_data[keystring + '-Eo'] = ((rhol - rhog)*g*((0.001*global_averages[:,0])**2.0))/sigma
    
            # modelling data  
            Re = (rhol*global_averages[:,1]*0.001*global_averages[:,0])/0.000935
            Eo = ((rhol - rhog)*g*((0.001*global_averages[:,0])**2.0))/sigma
            pNorm = p/(0.1*Eo)
            mdict['Re'] = Re
            mdict['Eo'] = Eo
            mdict['eg'] = eg*np.ones(nClasses)
            mdict['p_norm'] = pNorm
            mdict['Cdi'] = global_averages[:,2]
            mdf = pd.DataFrame(mdict)
            m_data.append(mdf)
        plt.xlabel("bubble diameter [ mm ]")
        plt.ylabel("Drag coefficient [ - ]")
        plt.xlim([0, 5])
        plt.ylim([0, 1.5])
        plt.legend()
    plt.show()
    # export all data to text files
    model_data = pd.concat(m_data, ignore_index=True)
    model_data.to_csv('model_data.txt', sep='\t', index=False, na_rep='nan')
    drag_data.to_csv('drag_data.txt', sep='\t', index=False, na_rep='nan')
    
    return model_data

# model function

In [ ]:
def model(beta2, beta3, beta4, Re, Eo, eg, p_norm):
    # single bubble drag coefficient at creeping flow conditions
    Cd0 = 24.0/Re
    
    # correction for inertial and buoyant effects
    # ReEo represents the relative magnitude of 
    # form and viscous drag
    fReEo = (1.0 + (Re*Eo)**beta2)
    
    # pressure correction
    m = beta3*(1.0 - np.exp(-beta4*p_norm))
    
    # swarm correction
    fswarm = (1.0 - eg)**m
    
    # all effect combined
    Cdi = Cd0*fReEo*fswarm
    
    return Cdi

# objective function

In [ ]:
def objfunc(betas):
    beta0, beta1, beta2, beta3, beta4 = betas[0], betas[1], betas[2], betas[3], betas[4]
    # modelling data
    xydata = modellingData(beta0, beta1)
    
    # drop rows with missing values
    xydata.dropna(inplace=True)
    xydata.reset_index(drop=True, inplace=True)
    
    # independent variables
    Re = xydata['Re']
    Eo = xydata['Eo']
    eg = xydata['eg']
    p_norm = xydata['p_norm']

    # dependent variable
    Cdi = xydata['Cdi']
    
    # predict drag coefficients
    Cdi_pred = model(beta2, beta3, beta4, Re, Eo, eg, p_norm)
    
    residual = (Cdi - Cdi_pred)
    
    return residual

# parameter estimation

In [ ]:
# least squares parameter estimation
# initial model parameters
betas_0 = np.array([0.163, 0.757, 0.46, 3.96, 0.07])

params = least_squares(objfunc, betas_0, jac='3-point', method='lm', loss='linear', verbose=2)

In [ ]:
xydata = plotModellingData(params.x[0], params.x[1])
Re = xydata['Re']
Eo = xydata['Eo']
eg = xydata['eg']
p_norm = xydata['p_norm']

Cdi = xydata['Cdi']

# model predictions
Cdi_pred = model(params.x[2], params.x[3], params.x[4], Re, Eo, eg, p_norm)

# estimate R^2
squared_res = (Cdi-Cdi_pred)**2
ssr = np.sum(squared_res)
sst = np.sum((Cdi - np.mean(Cdi))**2)
Rsq = 1.0 - (ssr/sst)

# estimate errors on fitted parameters
U, s, Vh = svd(params.jac)
tol = np.finfo(float).eps*s[0]*max(params.jac.shape)
w = s > tol
cov = (Vh[w].T/s[w]**2) @ Vh[w]  
perr = np.sqrt(np.diag(cov))/np.sqrt(len(Cdi)-1)  

print(f'beta0 = {params.x[0]:.2f} +/- {perr[0]:.2f}, beta1 = {params.x[1]:.2f} +/- {perr[1]:.2f}')
print(f'beta2 = {params.x[2]:.2f} +/- {perr[2]:.2f}, beta3 = {params.x[3]:.2f} +/- {perr[3]:.2f}, beta4 = {params.x[4]:.2f} +/- {perr[4]:.2f}')
print(f'R^2 = {Rsq:.2f}')

residuals = Cdi - Cdi_pred

# residual plot
plt.rcParams['figure.figsize'] = (6,5)
plt.plot(residuals, 'bo')
plt.xlabel('Observation')
plt.ylabel('residual')
plt.show()

# bubble size distribution

In [ ]:
# main directory
main_dir = os.getcwd()

# pressure folders
p_folder = [0.14, 0.46, 2, 4]

# gas holdup folders
eg_folder = [0.0987, 0.1873, 0.264, 0.3329]

# chord lengths for plotting cumulative distribution function
chords = np.linspace(0, 3, 1000)

# data frame for storing cumulative distribution functions
df_plot = pd.DataFrame(chords, columns=['Chord'])

# plotting
plt.figure(figsize= [12, 10])
lstyles = ['solid', 'solid', 'solid', 'solid']
lcolors = ['blue', 'red', 'black', 'green']

# loop counters
i = 0
for eg in eg_folder:
    plt.subplot(2,2, i+1)
    j=0
    for p in p_folder:
        path_to_files = main_dir + '/P = ' + str(p) + ' MPa' + '/eG = ' + str(eg)
        files = sorted(glob.glob(os.path.join(path_to_files,'*.txt')))
        list_df = []
        for file in files:
            list_df.append(pd.read_csv(file, delimiter='\t'))
        df = pd.concat(list_df)
        shape, loc, scale = lognorm.fit(df['Chord(mm)'])
        cdf = lognorm.cdf(chords, shape, loc, scale)
        pdf = lognorm.pdf(chords, shape, loc, scale)
        df_plot['cdf-P'+str(int(1000*p))+'-eG' + str(int(10000*eg))] = cdf
        df_plot['pdf-P'+str(int(1000*p))+'-eG' + str(int(10000*eg))] = pdf
        plt.plot(chords,cdf,linestyle=lstyles[j], color=lcolors[j], linewidth=2, alpha=1, label='P = ' + str(p_folder[j]) + ' MPa')
        j = j + 1
    plt.xlabel('Chord Length (mm)')
    plt.ylabel('Cumulative Distribution Function (-)')
    plt.text(0.6,0.1,r'$\varepsilon_g = $'+str(eg_folder[i]))
    plt.legend()
    i = i + 1
plt.show()
df_plot.to_csv('chord-length-cdf.txt', sep = '\t', index=False)